<a href="https://colab.research.google.com/github/maryamt95/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [20]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-18 02:48:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-12-18 02:48:36 (6.58 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon_Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [8]:
vine_vote=vine_df.filter(vine_df.total_votes>=20)


In [22]:
vine_vote.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R4PKAZRQJJX14 |1          |21           |34         |N   |N                |
|R2CI0Y288CC7E2|1          |21           |35         |N   |Y                |
|R127WEQY2FM1T3|1          |147          |175        |N   |Y                |
|R3EZ0EPYLDA34S|1          |14           |31         |N   |Y                |
|R2FJ94555FZH32|2          |55           |60         |N   |N                |
|R1U3AR67RE273L|1          |51           |65         |N   |Y                |
|R3PZOXA5X1U8KW|4          |31           |36         |N   |N                |
|R6KTC1OPIOIIG |2          |19           |34         |N   |Y                |
|R36O341WWXXKNP|5          |28           |31         |N   |N                |
|R3GSK9MM8DNOYI|1          |4            |32         |N   |N    

In [49]:
vine_updated=vine_vote.withColumn('rates',vine_vote['helpful_votes']/vine_vote['total_votes'])
vine_filtered=vine_updated.filter(vine_updated.rates>=0.5)
#paid
vine_filtered_y=vine_filtered.filter(vine_filtered.vine=='Y')
vine_filtered_y_total=vine_filtered_y.count()
vine_filtered_y_5=vine_filtered_y.filter(vine_filtered_y.star_rating==5).count()
vine_filtered_p=vine_filtered_y_5/vine_filtered_y_total

#unpaid
vine_filtered_n=vine_filtered.filter(vine_filtered.vine=='N')
vine_filtered_n_total=vine_filtered_n.count()
vine_filtered_n_5=vine_filtered_n.filter(vine_filtered_n.star_rating==5).count()
vine_filtered_n_p=vine_filtered_n_5/vine_filtered_n_total


In [50]:
print(f"total count of review as Vine program is {vine_filtered_y_total}.Out of which 5 star rating is {vine_filtered_y_5} which is {vine_filtered_p}%")

total count of review as Vine program is 94.Out of which 5 star rating is 48 which is 0.5106382978723404%


In [51]:
print(f"total count of reviews not part of Vine program is {vine_filtered_n_total}.Out of which 5 star rating is {vine_filtered_n_5} which is {vine_filtered_n_p}%")

total count of reviews not part of Vine program is 40471.Out of which 5 star rating is 15663 which is 0.38701786464381904%
